In [1]:
import pandas as pd
import numpy as np  #for text pre-processing
import re, string
from joblib import dump, load
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix, roc_curve, auc
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
df = pd.read_csv('../data/nrao_projects.csv')
#df['target'] = pd.get_dummies(df['fs_type'], dtype=int)['line']
df['text'] = df.project_title + ". " + df.project_abstract
df['text'] = df['text'].astype(str)
df.head()

,project_code,project_title,project_abstract,fs_type,target,text
0,2018.1.01205.L,Fifty AU STudy of the chemistry in the disk/en...,The huge variety of planetary systems discover...,line,1,Fifty AU STudy of the chemistry in the disk/en...
1,2022.1.00316.L,COMPASS: Complex Organic Molecules in Protosta...,The emergence of complex organic molecules in ...,line,1,COMPASS: Complex Organic Molecules in Protosta...
2,2017.1.00161.L,ALCHEMI: the ALMA Comprehensive High-resolutio...,A great variety in gas composition is observed...,line,1,ALCHEMI: the ALMA Comprehensive High-resolutio...
3,2021.1.01616.L,ALMA JELLY - Survey of Nearby Jellyfish and Ra...,We propose the first ever statistical survey o...,line,1,ALMA JELLY - Survey of Nearby Jellyfish and Ra...
4,2021.1.00869.L,Bulge symmetry or not? The hidden dynamics of ...,A radio survey of red giant SiO sources in the...,line,1,Bulge symmetry or not? The hidden dynamics of ...


In [3]:
adds = pd.DataFrame({
    'project_code':['noah_and_ryans cool test'],
    'project_title':['noah_and_ryans cool test'],
    'project_abstract':['noah_and_ryans cool test'],
    'fs_type':['noah_and_ryans cool test'],
    'target':[1],
    'text':['noah_and_ryans cool test']
})

In [4]:
adds

,project_code,project_title,project_abstract,fs_type,target,text
0,noah_and_ryans cool test,noah_and_ryans cool test,noah_and_ryans cool test,noah_and_ryans cool test,1,noah_and_ryans cool test


In [5]:
df2 = pd.concat([df,adds])
df2 = df2.reset_index()
df2.tail()

,index,project_code,project_title,project_abstract,fs_type,target,text
4524,4524,2018.1.01465.V,Disorder vs. Order: Discerning the nature of t...,We propose using a combined ALMA+GMVA array to...,line,1,Disorder vs. Order: Discerning the nature of t...
4525,4525,2021.1.00767.V,Ultra-high resolution imaging of 3C84,3C84 is a prime target for high angular resolu...,line,1,Ultra-high resolution imaging of 3C84. 3C84 is...
4526,4526,2019.1.00183.V,Imaging Jet and Magnetic Field near the Spinni...,Understanding the jet formation in supermassiv...,line,1,Imaging Jet and Magnetic Field near the Spinni...
4527,4527,2022.1.00710.V,First subparsec-scale imaging of the new TeV g...,Since the first detection of astrophysical gam...,line,1,First subparsec-scale imaging of the new TeV g...
4528,0,noah_and_ryans cool test,noah_and_ryans cool test,noah_and_ryans cool test,noah_and_ryans cool test,1,noah_and_ryans cool test


In [11]:
test_set = load('test_set.joblib')

In [6]:
#convert to lowercase, strip and remove punctuations
def preprocess(text):
    text = text.lower() 
    text = text.strip()  
    text = re.compile('<.*?>').sub('', text) 
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)  
    text = re.sub('\s+', ' ', text)  
    text = re.sub(r'\[[0-9]*\]',' ',text) 
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\d',' ',text) 
    text = re.sub(r'\s+',' ',text) 
    return text

In [7]:
df['clean_text'] = df['text'].apply(lambda x: preprocess(x))
df.head()

,project_code,project_title,project_abstract,fs_type,target,text,clean_text
0,2018.1.01205.L,Fifty AU STudy of the chemistry in the disk/en...,The huge variety of planetary systems discover...,line,1,Fifty AU STudy of the chemistry in the disk/en...,fifty au study of the chemistry in the disk en...
1,2022.1.00316.L,COMPASS: Complex Organic Molecules in Protosta...,The emergence of complex organic molecules in ...,line,1,COMPASS: Complex Organic Molecules in Protosta...,compass complex organic molecules in protostar...
2,2017.1.00161.L,ALCHEMI: the ALMA Comprehensive High-resolutio...,A great variety in gas composition is observed...,line,1,ALCHEMI: the ALMA Comprehensive High-resolutio...,alchemi the alma comprehensive high resolution...
3,2021.1.01616.L,ALMA JELLY - Survey of Nearby Jellyfish and Ra...,We propose the first ever statistical survey o...,line,1,ALMA JELLY - Survey of Nearby Jellyfish and Ra...,alma jelly survey of nearby jellyfish and ram ...
4,2021.1.00869.L,Bulge symmetry or not? The hidden dynamics of ...,A radio survey of red giant SiO sources in the...,line,1,Bulge symmetry or not? The hidden dynamics of ...,bulge symmetry or not the hidden dynamics of t...


In [8]:
in_tfidf = load('tfidf_vectorizer.joblib')
in_model = load('log_model.joblib')

In [9]:
tfidfs = in_tfidf.transform(df.clean_text)

In [10]:
in_model.predict(tfidfs).sum()

3731

In [19]:
# on test data from notebook....we want to see 770 ones predicted
in_model.predict(in_tfidf.transform(test_set)).sum()

770